In [ ]:
%%capture
!pip install dvc fastds

import os

### Set up local repo and branch

In [ ]:
# Clone repo with personal token (Settings -> Tokens -> Default Access Token)
!git clone https://{your_token}@dagshub.com/Omdena/NYU.git

# Create branch and make it track remote branch, then change to created branch
%cd NYU
!git branch task-4-training origin/task-4-training
!git checkout task-4-training

%cd tasks/task-4-language-transformer-models/workflow
print(f"\nCurrent directory: {os.getcwd()}")

### Set up remote DVC

In [ ]:
!dvc remote add origin --local https://dagshub.com/Omdena/NYU.dvc

In [ ]:
!dvc remote modify --local origin auth basic
!dvc remote modify --local origin user {your_username}
!dvc remote modify --local origin password {your_token}

### Pull artifacts (data, model files and predictions in test set)

In [ ]:
!dvc pull -r origin

### Install dependencies

In [ ]:
%%capture
!pip install -r requirements.txt

### Set mlflow credentials as env variables

In [ ]:
%env MLFLOW_TRACKING_URI=https://dagshub.com/Omdena/NYU.mlflow
%env MLFLOW_TRACKING_USERNAME={your_username}
%env MLFLOW_TRACKING_PASSWORD={your_token}

### Shouldn't be run. Creates and runs workflow. Used to create ```train_eval``` pipeline shown in the repo). Since it already exists in repo, will throw error.

In [ ]:
!dvc run -n train_eval \
-d ../data/hate-speech-homophobia/train_data.csv \
-d ../data/hate-speech-homophobia/dev_data.csv \
-d ../data/hate-speech-homophobia/test_data.csv \
-d params.yaml \
-o model_artifacts/model_best.pt \
-o model_artifacts/args.pt \
-o model_artifacts/test_labels.txt \
-o model_artifacts/test_labels_gold.txt \
-o model_artifacts/test_labels_pred.txt \
python train.py

### Run workflow

Modify `params.yaml` file to tune hyperparams and training arguments as needed. Then run training script with dvc to generate DAG in Dagshub.

In [ ]:
!rm -r model_artifacts/

In [ ]:
!dvc repro

Save changes

In [ ]:
!git add dvc.lock
!git commit -m ""
!git push

In [ ]:
!dvc push -r origin